In [ ]:
# IMPORT PACKAGES

In [26]:
import os
import nltk
import pandas as pd
import Step1_Module as s1_m

In [2]:
# IMPORT files

In [5]:
# Import Docket sheets w/ pre-classified time periods
df_docketsheet_master_file_preClass_timePeriods = pd.read_excel(r'C:\Users\Chris.Cirelli\Desktop\Python Programming Docs\GSU\Sprint Project\Docket-Sheet-Classification\DocketSheet Classification_70_02.22.2018.xlsx')


In [6]:
# Import List of Key Words
df_final_words = pd.read_excel(r'C:\Users\Chris.Cirelli\Desktop\Python Programming Docs\GSU\Sprint Project\Docket-Sheet-Classification\Results_top words\Final_List_key_words.xlsx')

In [7]:
def get_set_clean_words(df_key_words):
    List_final_words = []

    for col in df_final_words.columns:
        df_series = [word for word in df_final_words[col]]
        [List_final_words.append(word) for word in df_series]
    
    return set(List_final_words)
    

In [8]:
Set_key_words = get_set_clean_words(df_final_words)

In [ ]:
# IMPORT ORIGINAL DOCKET SHEET FILE W/ PRE-CLASSIFIED ROWS

In [9]:
def adjust_rows_cols_docketsheet(df_docket_sheet):
    
    df_docketsheet_preclassified_narrow_col = df_docket_sheet.iloc[:,:7]
    df_TP_int = df_docketsheet_preclassified_narrow_col['Time Period'] > 0
    df_docketsheet_preclassified_narrow_TP = df_docketsheet_preclassified_narrow_col[df_TP_int]
    
    return df_docketsheet_preclassified_narrow_TP 

df_docketsheet_adj_rows_cols = adjust_rows_cols_docketsheet(df_docketsheet_master_file_preClass_timePeriods)

In [ ]:
# INSPECT DATAFRAMES

In [10]:
df_docketsheet_adj_rows_cols.describe()

,Index,Relevant,Time Period
count,1295.000000,1294.000000,1295.000000
mean,31.795367,0.826121,4.936680
std,28.046423,0.379152,2.351757
min,0.000000,0.000000,1.000000
25%,12.000000,1.000000,3.000000
50%,25.000000,1.000000,5.000000
75%,42.500000,1.000000,5.000000
max,169.000000,1.000000,11.000000


In [15]:
def get_freq_dist(Docketsheet, list_keywords):
    '''The purpose of this function is to create a word frequency table for a set of documents for a set of key words
    
    Input  = Docketsheet: docket sheet entries + pre-classified life cycle stages 
             list_keywords = list of key words to match.  56 in total.  
    
    Operations:
             1.) Create an empty dataframe whose index is the list of key words.  There are 56 in total. 
             2.) Create a count of the rows
             3.) Iterate over the rows in the docket sheet dataframe as a list of tuples. 
             4.) Count the times the function iterates the row statement. 
             5.) Create a list to capture the matches for the 56 key-words for a single row. 
             6.) Important:  Because there are multiple rows for the same case, the number in the counter will need to be
                 added to the column name else the columns will get over writen in the df_DAT[col] function. 
    
    Output = 1.) A pandas dataframe whose rows constitute each row in the Docketsheet and whose columns are comprise of the 
             words in the key word list.  The data will be 1/0 depending on if the key words was a match or not. 
             2.) The final column of the dataframe will be the life cycle stage for each docket sheet entry. 
    '''
    
    # Create New Dataframe
    df_DAT = pd.DataFrame({}, index = list_keywords)
    
    # Count of Rows
    Count = 0

    # Iterate over row of the docketsheet df as an enumerated tuple. 
    for row in Docketsheet.itertuples():

        # Count rows
        Count += 1
        
        # List to capture each row.  List will reset on each iteration of the code. 
        List_word_matches_single_row = []
    
        # Clean row[4], which is the text column of the docketsheet. 
        clean_tokenize_row = s1_m.clean_andTokenize_text(row[4])                
    
        # So iterate every word in key_word_list for each rows 
        for word in list_keywords:
            
            # Check to see if the word is in the clean text
            if word in clean_tokenize_row:
                
                # If there is a match, append 1 to the list
                List_word_matches_single_row.append(1)
            else: 
                # If not, append 0
                List_word_matches_single_row.append(0)
        
        # Create a string from the counter to add to your column name. 
        Row_num = 'row' + str(Count) + ' ' 
        
        # Create a column in your new datafrmae to capture the case number and list of matches. 
        df_DAT[Row_num + row[1]] = List_word_matches_single_row        

    # Return our dataframe w/ an index as the list of key words, columns as each case. 
    
    return df_DAT

In [16]:
key_word_freq_docketsheet = get_freq_dist(df_docketsheet_adj_rows_cols, 
                                          Set_key_words)

In [20]:
key_word_freq_docketsheet.head(5)

,row1 1:10-cv-00007-JEC,row2 1:10-cv-00007-JEC,row3 1:10-cv-00007-JEC,row4 1:10-cv-00007-JEC,row5 1:10-cv-00007-JEC,row6 1:10-cv-00007-JEC,row7 1:10-cv-00007-JEC,row8 1:10-cv-00007-JEC,row9 1:10-cv-00007-JEC,row10 1:10-cv-00007-JEC,...,row1286 1:16-cv-00305-MHC,row1287 1:16-cv-00305-MHC,row1288 1:16-cv-00305-MHC,row1289 1:16-cv-00305-MHC,row1290 1:16-cv-00305-MHC,row1291 1:16-cv-00305-MHC,row1292 1:16-cv-00305-MHC,row1293 1:16-cv-00305-MHC,row1294 1:16-cv-00305-MHC,row1295 1:16-cv-00305-MHC
obtain,1,1,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
notic,0,0,0,0,0,1,1,1,0,0,...,1,0,0,0,0,0,0,0,1,0
drive,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
discoveri,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
number,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Transpose dataframe

In [21]:
df_final  = pd.DataFrame.transpose(key_word_freq_docketsheet)

In [22]:
df_final.head(2)

,obtain,notic,drive,discoveri,number,order,fee,report,final,brown,...,http,alan,hawk,file,websit,neddal,close,deni,stipul,summari
row1 1:10-cv-00007-JEC,1,0,0,0,1,0,0,0,0,0,...,1,0,0,1,1,0,0,0,0,0
row2 1:10-cv-00007-JEC,1,0,0,1,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0


In [ ]:
# Append Life Cycle Stage to Dataframe

In [23]:
List_Life_Cycles = [x for x in df_docketsheet_adj_rows_cols['Time Period']]

df_final['Life Cycle Stage'] = List_Life_Cycles



In [24]:
df_final.head(2)

,obtain,notic,drive,discoveri,number,order,fee,report,final,brown,...,alan,hawk,file,websit,neddal,close,deni,stipul,summari,Life Cycle Stage
row1 1:10-cv-00007-JEC,1,0,0,0,1,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,1.0
row2 1:10-cv-00007-JEC,1,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,2.0


In [27]:
s1_m.write_to_excel(df_final, 'Data Analytics Table_WordMatch_All_Docket_Entries_v2')